/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

SELECT  `name` ,  `membercost` 
FROM  `Facilities` 
WHERE  `membercost` >0

/* Q2: How many facilities do not charge a fee to members? */

SELECT COUNT(  `name` ) 
FROM  `Facilities` 
WHERE  `membercost` =0

COUNT = 4


/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

SELECT  `facid` ,  `name` ,  `membercost` ,  `monthlymaintenance` 
FROM  `Facilities` 
WHERE  `membercost` >0
AND  `membercost` <  `monthlymaintenance` * 0.2



/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

SELECT * FROM `Facilities` WHERE `facid` in (1, 5)


/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

SELECT  `name` ,  `monthlymaintenance` , 
CASE WHEN  `monthlymaintenance` <100
THEN  'Cheap'
ELSE  'Expensive'
END AS cost
FROM  `Facilities` 


/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

SELECT `firstname`, `surname` FROM `Members` WHERE `joindate` = (SELECT MAX(`joindate`) FROM `Members`)


/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

SELECT DISTINCT CONCAT(  `firstname` ,  ' ',  `surname` ) , 
CASE WHEN  `facid` =0
THEN  'Tennis Court 1'
ELSE  'Tennis Court 2'
END AS name
FROM  `Members` 
INNER JOIN  `Bookings` 
USING (  `memid` ) 
WHERE  `facid` 
IN ( 0, 1 ) 
AND  `firstname` <>  'GUEST'
ORDER BY  `firstname`

/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

SELECT CONCAT(  `firstname` ,  ' ',  `surname` ) , name,
CASE WHEN `memid` = 0 AND `slots` * `guestcost` > 30 THEN `slots` * `guestcost` WHEN `memid` <> 0 AND `slots` * `membercost` > 30 THEN `slots` * `membercost` END AS cost
FROM `Members` 
LEFT JOIN `Bookings` 
USING (  `memid` )
LEFT JOIN `Facilities` 
USING (  `facid` )
WHERE `starttime` LIKE '2012-09-14%'
HAVING cost > 30
ORDER BY cost DESC

/* Q9: This time, produce the same result as in Q8, but using a subquery. */

SELECT CONCAT(  `firstname` ,  ' ',  `surname` ) , name,
CASE WHEN `memid` = 0 AND `slots` * `guestcost` > 30 THEN `slots` * `guestcost` WHEN `memid` <> 0 AND `slots` * `membercost` > 30 THEN `slots` * `membercost` END AS cost
FROM `Members` 
LEFT JOIN `Bookings` 
USING (  `memid` )
LEFT JOIN `Facilities` 
USING (  `facid` )
WHERE `bookid` IN (SELECT `bookid` FROM `Bookings` WHERE `starttime` LIKE '2012-09-14%')
HAVING cost > 30
ORDER BY cost DESC


/* PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

QUESTIONS:
/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */


SELECT `name`, 
SUM(CASE WHEN `memid` = 0 THEN `slots` * `guestcost` ELSE `slots` * `membercost` END) AS revenue
FROM `Facilities` 
LEFT JOIN `Bookings` 
USING(`facid`)
GROUP BY `facid`
HAVING revenue < 1000
ORDER BY revenue DESC

In [385]:
import pandas as pd
import sqlite3
os.chdir('/Users/josevans/Downloads')
conn = sqlite3.connect("data.sqlite")


In [386]:
cur = conn.cursor()

In [387]:
cur.execute("SELECT surname, firstname, CASE WHEN recommendedby = NULL THEN 'N/A' ELSE (SELECT surname FROM Members WHERE memid = recommendedby) END AS RecommendedBy FROM Members ORDER BY surname")

In [388]:
results = cur.fetchall()
print(results)

[('Bader', 'Florence', None), ('Baker', 'Anne', None), ('Baker', 'Timothy', None), ('Boothe', 'Tim', None), ('Butters', 'Gerald', None), ('Coplin', 'Joan', None), ('Crumpet', 'Erica', None), ('Dare', 'Nancy', None), ('Farrell', 'Jemima', None), ('Farrell', 'David', None), ('GUEST', 'GUEST', None), ('Genting', 'Matthew', None), ('Hunt', 'John', None), ('Jones', 'David', None), ('Jones', 'Douglas', None), ('Joplette', 'Janice', None), ('Mackenzie', 'Anna', None), ('Owen', 'Charles', None), ('Pinker', 'David', None), ('Purview', 'Millicent', None), ('Rownam', 'Tim', None), ('Rumney', 'Henrietta', None), ('Sarwin', 'Ramnaresh', None), ('Smith', 'Darren', None), ('Smith', 'Tracy', None), ('Smith', 'Jack', None), ('Smith', 'Darren', None), ('Stibbons', 'Ponder', None), ('Tracy', 'Burton', None), ('Tupperware', 'Hyacinth', None), ('Worthington-Smyth', 'Henry', None)]


In [389]:
dfm = pd.read_sql_query("SELECT * FROM Members", conn)
dff = pd.read_sql_query("SELECT * FROM Facilities", conn)
dfb = pd.read_sql_query("SELECT * FROM Bookings", conn)

In [390]:
cur.close()
conn.close()

In [391]:
dfb.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [392]:
dff.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [393]:
dfm = dfm.set_index('memid')
dfm.head(35)

,surname,firstname,address,zipcode,telephone,recommendedby,joindate
memid,,,,,,,
0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05
5,Butters,Gerald,"1065 Huntingdon Avenue, Boston",56754,(844) 078-4130,1,2012-07-09 10:44:09
6,Tracy,Burton,"3 Tunisia Drive, Boston",45678,(822) 354-9973,,2012-07-15 08:52:55
7,Dare,Nancy,"6 Hunting Lodge Way, Boston",10383,(833) 776-4001,4,2012-07-25 08:59:12
8,Boothe,Tim,"3 Bloomsbury Close, Reading, 00234",234,(811) 433-2547,3,2012-07-25 16:02:35


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */


In [394]:
import numpy as np
bookings = dfb.copy()
members = dfm.copy()
facilities = dff.copy()
dfm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31 entries, 0 to 37
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   surname        31 non-null     object
 1   firstname      31 non-null     object
 2   address        31 non-null     object
 3   zipcode        31 non-null     object
 4   telephone      31 non-null     object
 5   recommendedby  31 non-null     object
 6   joindate       31 non-null     object
dtypes: object(7)
memory usage: 1.9+ KB


In [395]:
dfm.recommendedby.replace('', 0, inplace=True)

In [357]:
keys = [*dfm.index.values]
values = [dfm.firstname+' '+dfm.surname]
mem_dict=dict(zip(keys, values))
mem_dict = dict(mem_dict['0'])

In [396]:
dfm['rec_by'] = pd.DataFrame([mem_dict[str(row)] if row != 0 else 'N/A' for row in dfm.recommendedby], index=dfm.index)


In [397]:
dfm = dfm.sort_values(['surname', 'firstname'])
dfm.head(30)

,surname,firstname,address,zipcode,telephone,recommendedby,joindate,rec_by
memid,,,,,,,,
15,Bader,Florence,"264 Ursula Drive, Westford",84923,(833) 499-3527,9,2012-08-10 17:52:03,Ponder Stibbons
12,Baker,Anne,"55 Powdery Street, Boston",80743,844-076-5141,9,2012-08-10 14:23:22,Ponder Stibbons
16,Baker,Timothy,"329 James Street, Reading",58393,833-941-0824,13,2012-08-15 10:34:25,Jemima Farrell
8,Boothe,Tim,"3 Bloomsbury Close, Reading, 00234",234,(811) 433-2547,3,2012-07-25 16:02:35,Tim Rownam
5,Butters,Gerald,"1065 Huntingdon Avenue, Boston",56754,(844) 078-4130,1,2012-07-09 10:44:09,Darren Smith
22,Coplin,Joan,"85 Bard Street, Bloomington, Boston",43533,(822) 499-2232,16,2012-08-29 08:32:41,Timothy Baker
36,Crumpet,Erica,"Crimson Road, North Reading",75655,(811) 732-4816,2,2012-09-22 08:36:38,Tracy Smith
7,Dare,Nancy,"6 Hunting Lodge Way, Boston",10383,(833) 776-4001,4,2012-07-25 08:59:12,Janice Joplette
28,Farrell,David,"437 Granite Farm Road, Westford",43532,(855) 755-9876,0,2012-09-15 08:22:05,N/A


/* Q12: Find the facilities with their usage by member, but not guests */

In [164]:
dff.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [399]:
dfb.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [435]:
dfbMem = dfb[dfb.memid != '0'][['facid', 'memid']]

In [436]:
dfbMem = pd.DataFrame(dfbMem.groupby('memid')['facid'].apply(lambda x: list(np.unique(x))))
dfbMem

,facid
memid,
1,"[1, 2, 3, 4, 6, 7, 8]"
10,"[0, 1, 2, 3, 4, 5, 6, 7, 8]"
11,"[0, 1, 2, 3, 4, 5, 6, 7, 8]"
12,"[0, 1, 2, 3, 4, 5, 6, 8]"
13,"[0, 1, 2, 3, 4, 6, 7, 8]"
14,"[0, 1, 2, 3, 4, 5, 6, 7, 8]"
15,"[0, 1, 2, 3, 5, 6, 7, 8]"
16,"[0, 1, 2, 3, 4, 6, 8]"
17,"[0, 2, 3, 4, 6, 7, 8]"


In [404]:
dfbMem = dfbMem.reset_index()
dfbMem.replace({'memid': mem_dict}, inplace=True)

In [405]:
dfbMem

,memid,facid
0,Darren Smith,"[1, 2, 3, 4, 6, 7, 8]"
1,Charles Owen,"[0, 1, 2, 3, 4, 5, 6, 7, 8]"
2,David Jones,"[0, 1, 2, 3, 4, 5, 6, 7, 8]"
3,Anne Baker,"[0, 1, 2, 3, 4, 5, 6, 8]"
4,Jemima Farrell,"[0, 1, 2, 3, 4, 6, 7, 8]"
5,Jack Smith,"[0, 1, 2, 3, 4, 5, 6, 7, 8]"
6,Florence Bader,"[0, 1, 2, 3, 5, 6, 7, 8]"
7,Timothy Baker,"[0, 1, 2, 3, 4, 6, 8]"
8,David Pinker,"[0, 2, 3, 4, 6, 7, 8]"
9,Tracy Smith,"[0, 1, 2, 3, 4, 6, 7, 8]"


/* Q13: Find the facilities usage by month, but not guests */

In [439]:
dfbMem2 = dfb[dfb.memid != '0'][['facid', 'memid', 'starttime']]
dfbMem2.starttime = pd.to_datetime(dfbMem2.starttime).dt.month
dfbMem2

,facid,memid,starttime
0,3,1,7
1,4,1,7
3,7,1,7
4,8,1,7
5,8,1,7
...,...,...,...
4038,8,29,9
4039,8,29,9
4040,8,21,9
4041,8,16,9


In [447]:
dfbMem2grouped = pd.DataFrame(dfbMem2.groupby('starttime')['facid'].apply(lambda x: list(np.unique(x))))
dfbMem2grouped

,facid
starttime,
7,"[0, 1, 2, 3, 4, 5, 6, 7, 8]"
8,"[0, 1, 2, 3, 4, 5, 6, 7, 8]"
9,"[0, 1, 2, 3, 4, 5, 6, 7, 8]"


In [452]:
dfbMem2grouped2 = pd.DataFrame(dfbMem2.groupby(['starttime', 'facid'])['facid'].apply(lambda x: len(x)))
dfbMem2grouped2 = dfbMem2grouped2.rename(columns={'facid':'Num_bookings_per_month'})
dfbMem2grouped2

Num_bookings_per_month
starttime facid                        
7         0                          65
          1                          41
          2                          51
          3                          48
          4                          77
          5                           4
          6                          23
          7                          68
          8                         103
8         0                         111
          1                         109
          2                         132
          3                         143
          4                         153
          5                           9
          6                          85
          7                         154
          8                         272
9         0                         132
          1                         126
          2                         161
          3                         194
          4                         191
          5                          14
          6                          87
          7                         199
          8                         408